In [6]:
# Load and prepare data
import pandas as pd

df = pd.read_csv('../Data/RADCURE_Clinical_v04_20241219.csv')

In [ ]:
from lifelines import KaplanMeierFitter
from lifelines.statistics import logrank_test
import matplotlib.pyplot as plt

# Filter and clean
df_km1 = df_proc.copy()
df_km1 = df_km1[df_km1['Tx Modality'].notna()]

# Plot
plt.figure(figsize=(10, 6))
kmf = KaplanMeierFitter()

for group in df_km1['Tx Modality'].unique():
    mask = df_km1['Tx Modality'] == group
    kmf.fit(df_km1.loc[mask, 'Length FU'], event_observed=df_km1.loc[mask, 'event'], label=group)
    kmf.plot_survival_function()

plt.title('Kaplan-Meier Survival by Treatment Modality')
plt.xlabel('Follow-up Time (years)')
plt.ylabel('Survival Probability')
plt.grid(True)
plt.legend()
plt.show()

# Optional: Log-rank test
group1 = df_km1[df_km1['Tx Modality'] == 'RT alone']
group2 = df_km1[df_km1['Tx Modality'] == 'ChemoRT']

results1 = logrank_test(group1['Length FU'], group2['Length FU'],
                        event_observed_A=group1['event'],
                        event_observed_B=group2['event'])
print("Log-rank p-value (RT alone vs ChemoRT):", results1.p_value)


In [ ]:
# Filter and clean
df_km2 = df_proc.copy()
df_km2 = df_km2[df_km2['Stage'].notna()]

# Plot
plt.figure(figsize=(10, 6))
kmf = KaplanMeierFitter()

for group in df_km2['Stage'].unique():
    mask = df_km2['Stage'] == group
    kmf.fit(df_km2.loc[mask, 'Length FU'], event_observed=df_km2.loc[mask, 'event'], label=group)
    kmf.plot_survival_function()

plt.title('Kaplan-Meier Survival by Stage')
plt.xlabel('Follow-up Time (years)')
plt.ylabel('Survival Probability')
plt.grid(True)
plt.legend()
plt.show()

# Optional: Log-rank test (example: IVA vs IVB)
group1 = df_km2[df_km2['Stage'] == 'IVA']
group2 = df_km2[df_km2['Stage'] == 'IVB']

results2 = logrank_test(group1['Length FU'], group2['Length FU'],
                        event_observed_A=group1['event'],
                        event_observed_B=group2['event'])
print("Log-rank p-value (Stage IVA vs IVB):", results2.p_value)


In [ ]:
from lifelines import CoxPHFitter
import pandas as pd

# Select covariates explicitly
covariates = ['Age', 'Stage', 'Tx Modality', 'Chemo']  # 4 included here
df_cox = df_proc[['Length FU', 'event'] + covariates].dropna().copy()

# One-hot encode categorical covariates
df_cox_encoded = pd.get_dummies(df_cox, columns=['Stage', 'Tx Modality', 'Chemo'], drop_first=True)

# Fit Cox model
cph = CoxPHFitter()
cph.fit(df_cox_encoded, duration_col='Length FU', event_col='event')

# Print model results
cph.print_summary()

# Check PH assumption
cph.check_assumptions(df_cox_encoded, p_value_threshold=0.05)


In [ ]:
from sksurv.ensemble import RandomSurvivalForest
from sksurv.metrics import concordance_index_censored
from sklearn.model_selection import train_test_split

# Prepare data
y_rsf = df_model_encoded[['event', 'Length FU']].copy()
y_rsf_struct = y_rsf.to_records(index=False)
X_rsf = df_model_encoded.drop(columns=['event', 'Length FU'])

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_rsf, y_rsf_struct, test_size=0.2, random_state=42)

# Train RSF model
rsf = RandomSurvivalForest(n_estimators=100, min_samples_split=10, min_samples_leaf=15,
                           max_features='sqrt', n_jobs=-1, random_state=42)
rsf.fit(X_train, y_train)

# Evaluate RSF
c_index_rsf = rsf.score(X_test, y_test)
print("RSF Concordance Index:", round(c_index_rsf, 3))

# Compare to Cox C-index
from sksurv.metrics import concordance_index_censored
cox_c_index = concordance_index_censored(y_test['event'], y_test['Length FU'],
                                         cph.predict_partial_hazard(X_test))[0]
print("Cox Concordance Index:", round(cox_c_index, 3))

# Variable importance
import pandas as pd
importances = pd.Series(rsf.feature_importances_, index=X_rsf.columns).sort_values(ascending=False)
print("Top RSF Features:")
print(importances.head(10))
